<a href="https://colab.research.google.com/github/dp22acn/Time_series_modelling/blob/main/Assignment_Rework_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Making Initial Plots**

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import warnings
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.stattools import adfuller
import os
import pathlib
from google.colab import drive

# Install pmdarima
!pip install pmdarima

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = [10, 7.5]

# Mount Google Drive
drive.mount('/content/drive')

# Load Johnson & Johnson dataset
file_path = "/content/drive/MyDrive/jj.csv"
data = pd.read_csv(file_path)

# Display the first few rows and data information
print(data.head())
print(data.info())

# Plot Johnson & Johnson Quarterly Sales
data['data'].plot(title='Johnson & Johnson Quarterly Sales', figsize=(10, 6))
plt.ylabel('Sales')
plt.xlabel('Year')
plt.show()


# **Perform all the time series analysis tasks to test for non-stationarity**

In [ ]:
# Plotting Auto Correlation Function (ACF) and Partial Auto Correlation Function (PACF)
plot_acf(data['data'], color='green')
plt.title('Auto Correlation Function (ACF)')
plt.xlabel('Lags')
plt.ylabel('ACF')
plt.show()

plot_pacf(data['data'], color='green')
plt.title('Partial Auto Correlation Function (PACF)')
plt.xlabel('Lags')
plt.ylabel('PACF')
plt.show()

# Augmented Dickey-Fuller Test (ADF) for stationarity check
result = adfuller(data['data'])
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))


# **Make the data stationary**

In [ ]:
# Take the logarithm of the data and difference it to make it stationary
data['log_data'] = np.log(data['data'])
data['diff_log_data'] = data['log_data'].diff().dropna()

# Plot the differenced data
data['diff_log_data'].plot(title='Differenced Logarithm of Johnson & Johnson Sales', figsize=(10, 6))
plt.ylabel('Differenced Log Sales')
plt.xlabel('Year')
plt.show()

# Check stationarity again with ADF test
result = adfuller(data['diff_log_data'].dropna())
print('ADF Statistic (Differenced Log):', result[0])
print('p-value:', result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))


# **Define an ARMA model**

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pmdarima as pm

# Define the ARMA model with pmdarima to find best parameters
auto_model = pm.auto_arima(data['diff_log_data'].dropna(), start_p=0, start_q=0,
                           max_p=8, max_q=8, seasonal=False,
                           d=0, trace=True, error_action='ignore',
                           suppress_warnings=True, stepwise=True)

print("Best AIC (Auto):", auto_model.aic())
print(auto_model.summary())


# **Find the best model parameters: p,d,q**

In [ ]:
# Manual approach to find the best SARIMA model
min_aic = float('inf')
best_order = None

for p, d, q in product(range(9), range(2), range(9)):
    try:
        model = SARIMAX(data['data'], order=(p, d, q))
        results = model.fit(disp=False)
        if results.aic < min_aic:
            min_aic = results.aic
            best_order = (p, d, q)
    except:
        continue

print("Best AIC (Manual):", min_aic)
print("Best Order (Manual):", best_order)


# **Test model performance**

In [ ]:
# Create and fit the best SARIMA model based on manual approach
best_model = SARIMAX(data['data'], order=best_order)
best_model_fit = best_model.fit()

# Display the summary of the best model
print("Best Model Summary:")
print(best_model_fit.summary())

# Plot diagnostics for the best model
best_model_fit.plot_diagnostics()
plt.show()


# **Forecast model 24 months into the future**

In [ ]:
# Forecast for the next 24 months
n_forecast = 24
forecast_result = best_model_fit.get_forecast(steps=n_forecast)

# Get forecasted mean and confidence intervals
forecasted_mean = forecast_result.predicted_mean
confidence_intervals = forecast_result.conf_int()

# Create a new index for the forecasted values
forecast_index = np.arange(len(data['data']), len(data['data']) + n_forecast)

# Plot the actual data, forecasted values, and confidence intervals
plt.figure(figsize=(12, 8))
plt.plot(data['data'], label='Actual Sales', color='blue')
plt.plot(forecast_index, forecasted_mean, label='Forecast', color='orange')
plt.fill_between(forecast_index, confidence_intervals.iloc[:, 0], confidence_intervals.iloc[:, 1], color='green', alpha=0.5, label='Confidence Interval')
plt.title('Forecast of Johnson & Johnson Sales with Confidence Intervals')
plt.xlabel('Time')
plt.ylabel('Sales')
plt.legend()
plt.show()


# **Build an RNN based model (LSTM, GRU)**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load Johnson & Johnson dataset
file_path = "/content/drive/MyDrive/jj.csv"
data = pd.read_csv(file_path)

# Preprocess data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data['data'].values.reshape(-1, 1))

# Split data into train and test sets (70-30 split)
train_size = int(len(scaled_data) * 0.7)
train_data, test_data = scaled_data[:train_size], scaled_data[train_size:]

# Define function to create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

# Define sequence length
seq_length = 12

# Create sequences for training and testing data
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

# Build the GRU model
model = Sequential([
    GRU(64, activation='relu', input_shape=(seq_length, 1)),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# Forecasting
train_forecast = model.predict(X_train)
test_forecast = model.predict(X_test)

# Inverse transform forecasted values
train_forecast = scaler.inverse_transform(train_forecast).flatten()
test_forecast = scaler.inverse_transform(test_forecast).flatten()
y_train = scaler.inverse_transform(y_train).flatten()
y_test = scaler.inverse_transform(y_test).flatten()

# Calculate RMSE
train_rmse = np.sqrt(mean_squared_error(y_train, train_forecast))
test_rmse = np.sqrt(mean_squared_error(y_test, test_forecast))
print(f'Train RMSE: {train_rmse}')
print(f'Test RMSE: {test_rmse}')

# Plotting the forecast
plt.figure(figsize=(12, 6))

# Plot actual train sales
plt.plot(data.index[seq_length:seq_length + len(train_forecast)], y_train, label='Actual Train Sales', color='blue')

# Plot actual test sales (starting after the train data)
plt.plot(data.index[train_size + seq_length:], y_test, label='Actual Test Sales', color='green')

# Plot train forecast
plt.plot(data.index[seq_length:seq_length + len(train_forecast)], train_forecast, label='Train Forecast', color='red', linestyle='--')

# Plot test forecast (starting after the train data)
plt.plot(data.index[train_size + seq_length:], test_forecast, label='Test Forecast', color='orange', linestyle='--')

plt.title('Forecast of Johnson & Johnson Sales with GRU')
plt.xlabel('Time')
plt.ylabel('Sales')
plt.legend()
plt.show()
